### Goal: Merge the data sets below into one clean data set that can be used to basic exploratory data analysis.

## Data Sets Used

**Fire Incidents (2010-2017)**
Catalogues all fire incidents to which the PFD was called. Why: Fire Incidents are what we will be trying to predict and therefore will be used to evaluate the accuracy of our model.
Peoria Fire Department

**Fire Inspections
(2010-2017)**
Catalogues inspection results for non-residential buildings Why: How long ago an inspection occured may impact the risk of a fire in the near future.
Peoria Fire Department

**Building Characteristics**
Catalogues several characteristics of all buildings, such as year built, square footage, etc. Why: Some building characteristics may impact the risk of a fire in the future. For example, older buildings may contain knob-and-tube electrical wiring, which is less safe than modern electrical wiring.
Peoria County GIS Department / City of Peoria Information Systems Department

_Optional_

**Code Violations**
(Dates?)
Catalogues ordinance violations found when inspecting buildings Why: Code violations may point to issues that could cause a fire, such as exposed electrical wiring.
City of Peoria Community Development Department

## My thought process
We have 3 main data sets that we'll be using for the exploratory data analysis (EDA) but they all have a variety of column names. In addition, our inspections data is split between 2010-2016 data and 2017 data, in spreadsheets that have different columns.

Names of the datasets that we're working with:
1. roland_fire_join.csv - commercial buildings in the city with firezone id and firestation
2. addresses_commercialbuildings_peoria.csv - characteristics of commercial buildings in Peoria
3. incidents_of_fire_2010_to_2017.csv - incidents of fire between the years of 2010-2017
4. inspection_data_2017.csv - fire inspections in 2017
5. inspections_data_before_2017.csv - fire inspections from 2010 to 2016
